In [2]:
import pandas as pd
import glob
import os

# 1. Identify all CSV files (both *Extent*.csv and *Area*.csv)
path_to_data = "data"
all_csv_files = glob.glob(os.path.join(path_to_data, "*.csv"))

# 2. Prepare a list to collect tidied data
tidy_dfs = []

# 3. Loop over each CSV file, detect if it is "Area" or "Extent",
#    parse the region name, then reshape to long format
for file_path in all_csv_files:
    filename = os.path.basename(file_path)

    # Detect whether this is area or extent:
    # e.g. "N_Sea_Ice_Index_Regional_Monthly_Data_G02135_v3.0 - Baffin-Area-km^2.csv"
    # vs  "N_Sea_Ice_Index_Regional_Monthly_Data_G02135_v3.0 - Baffin-Extent-km^2.csv"
    if "Area" in filename:
        metric = "area"
    elif "Extent" in filename:
        metric = "extent"
    else:        
        continue

    # Extract the region name from the file name
    # Example file name format:
    #   "N_Sea_Ice_Index_Regional_Monthly_Data_G02135_v3.0 - Baffin-Area-km^2.csv"
    # We split on " - " and take the second chunk, then split on "-" again:
    #   "Baffin-Area-km^2.csv" --> first chunk "Baffin"
    region_name = filename.split(" - ")[1].split("-")[0]

    # 4. Read the CSV.  Often the first 2 rows or so are metadata/headers.    
    df_raw = pd.read_csv(file_path, skiprows=2, header=None)

    # 5. Rename columns.  Typically we have 25 columns:
    #    - 1 for "Year"
    #    - then 12 pairs of (value, rank) for Jan–Dec
    col_names = [
        "Year",
        "Jan", "JanRank",
        "Feb", "FebRank",
        "Mar", "MarRank",
        "Apr", "AprRank",
        "May", "MayRank",
        "Jun", "JunRank",
        "Jul", "JulRank",
        "Aug", "AugRank",
        "Sep", "SepRank",
        "Oct", "OctRank",
        "Nov", "NovRank",
        "Dec", "DecRank"
    ]
    
    # If the file sometimes has fewer columns, slice col_names to match.
    df_raw.columns = col_names[:len(df_raw.columns)]

    # 6. Keep only the columns with the actual area/extent values (not ranks)
    keep_cols = ["Year", "Jan", "Feb", "Mar", "Apr", "May", 
                 "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    df_sub = df_raw[keep_cols]

    # 7. Convert wide to long (one row per Year‐Month)
    df_long = pd.melt(
        df_sub,
        id_vars="Year", 
        var_name="Month", 
        value_name="Value"
    )

    # 8. Add columns for Region and Metric
    df_long["Region"] = region_name
    df_long["Metric"] = metric

    # 9. Append to our list
    tidy_dfs.append(df_long)

# 10. Concatenate everything
df_tidy = pd.concat(tidy_dfs, ignore_index=True)

# 11. Drop rows with missing Year or Value
df_tidy.dropna(subset=["Year", "Value"], inplace=True)

# 12. Convert Year to integer (if read as float)
df_tidy["Year"] = df_tidy["Year"].astype(int)

# 13. Map month names to numeric (1–12)
month_map = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
    "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}
df_tidy["MonthNum"] = df_tidy["Month"].map(month_map)

# 14. Create a date column (1st day of each month)
df_tidy["Date"] = pd.to_datetime(dict(year=df_tidy["Year"], month=df_tidy["MonthNum"], day=1))

# 15. Reorder columns nicely
df_tidy = df_tidy[["Region", "Year", "Month", "MonthNum", "Date", "Metric", "Value"]]

# 16. Save to CSV
df_tidy.to_csv("combined_sea_ice_area_extent.csv", index=False)

print("Tidy dataset shape:", df_tidy.shape)
print(df_tidy.head(20))

Tidy dataset shape: (15427, 7)
           Region  Year Month  MonthNum       Date  Metric        Value
2   East_Siberian  1979   Jan         1 1979-01-01  extent  1303711.475
3   East_Siberian  1980   Jan         1 1980-01-01  extent  1303711.475
4   East_Siberian  1981   Jan         1 1981-01-01  extent  1303711.475
5   East_Siberian  1982   Jan         1 1982-01-01  extent  1303711.475
6   East_Siberian  1983   Jan         1 1983-01-01  extent  1303711.475
7   East_Siberian  1984   Jan         1 1984-01-01  extent  1303711.475
8   East_Siberian  1985   Jan         1 1985-01-01  extent  1303711.475
9   East_Siberian  1986   Jan         1 1986-01-01  extent  1303711.475
10  East_Siberian  1987   Jan         1 1987-01-01  extent  1303711.475
12  East_Siberian  1989   Jan         1 1989-01-01  extent  1303711.475
13  East_Siberian  1990   Jan         1 1990-01-01  extent  1303690.946
14  East_Siberian  1991   Jan         1 1991-01-01  extent  1303711.475
15  East_Siberian  1992   Jan    